# LAB 5 - Quantum Key Distribution

| Name | Surname | ID Number |
|------|---------|---------------------|
| Lorenzo | Calandra Buonaura | 2107761 |

**Import zone**

In [1]:
import numpy as np

In [2]:
def dataset_loading(file_path, t):
  state_mapping = {
    '00': 'H',
    '01': 'V',
    '10': 'D',
    '11': 'A'
  }

  decoy_mapping = {
    '00': 'N',
    '01': 'L',
    '10': 'H', 
    '11': 'X'
  }
  
  if t == "state":
    mapping = state_mapping
  elif t == "decoy":
    mapping = decoy_mapping
  else:
    raise ValueError(f"Invalid type: '{t}'. Accepted types are 'state' or 'decoy'.")
  
  with open(file_path, "rb") as file:
    states = []
  
    while byte := file.read(1):  # Read one byte at a time
      # Convert the byte to its binary representation
      binary_representation = bin(ord(byte))[2:].zfill(8)

      # Extract the last 2 bits
      state_bits = binary_representation[-2:]

      # Map the bits to their corresponding state
      state = mapping.get(state_bits, "Unknown")

      states.append(state)
      
  return np.array(states)

In [3]:
transmitted_states_1 = dataset_loading("./Data/set1/states.txt", t="state")

In [4]:
recieved_states_1 = dataset_loading("./Data/set1/statesRCV.txt", t="state")

In [59]:
decoy_1 = dataset_loading("./Data/set1/decoy.txt", t="decoy")

In [5]:
states_trans_D_1 = transmitted_states_1[transmitted_states_1 == 'D']
states_trans_H_1 = transmitted_states_1[transmitted_states_1 == 'H']
states_trans_V_1 = transmitted_states_1[transmitted_states_1 == 'V']

In [6]:
prob_enc_X = len(states_trans_D_1) / len(transmitted_states_1)
prob_enc_Z = (len(states_trans_H_1) + len(states_trans_V_1)) / len(transmitted_states_1)

print(f'Probability of encoding in base X = {prob_enc_X}')
print(f'Probability of encoding in base Z = {prob_enc_Z}')
print(f'Total probability = {prob_enc_Z + prob_enc_X}')

Probability of encoding in base X = 0.10213526155021001
Probability of encoding in base Z = 0.89786473844979
Total probability = 1.0


In [7]:
states_rec_D_1 = recieved_states_1[recieved_states_1 == 'D']
states_rec_H_1 = recieved_states_1[recieved_states_1 == 'H']
states_rec_V_1 = recieved_states_1[recieved_states_1 == 'V']
states_rec_A_1 = recieved_states_1[recieved_states_1 == 'A']

In [8]:
prob_dec_X = (len(states_rec_D_1) + len(states_rec_A_1)) / len(recieved_states_1)
prob_dec_Z = (len(states_rec_H_1) + len(states_rec_V_1)) / len(recieved_states_1)

print(f'Probability of encoding in base X = {prob_dec_X}')
print(f'Probability of encoding in base Z = {prob_dec_Z}')
print(f'Total probability = {prob_dec_Z + prob_dec_X}')

Probability of encoding in base X = 0.5327614547537228
Probability of encoding in base Z = 0.4672385452462772
Total probability = 1.0


In [13]:
len(recieved_states_1) == len(transmitted_states_1)

True

In [21]:
def cond_prob(transmitted_states, received_states, a, b):
  mask_a = list(transmitted_states == a)
  rec_a = received_states[mask_a]
  rec_b_a = rec_a[rec_a == b]
  
  conditional_probability = len(rec_b_a) / len(rec_a)
  
  return conditional_probability

In [22]:
a_values = ['D', 'H', 'V']
b_values = ['D', 'H', 'V', 'A']

cond_probs = {}

for a in a_values:
  for b in b_values:
    cond_probs[(b, a)] = cond_prob(transmitted_states_1, recieved_states_1, a, b)

In [23]:
cond_probs

{('D', 'D'): 0.5463499409795984,
 ('H', 'D'): 0.2444528766721279,
 ('V', 'D'): 0.20121610999944858,
 ('A', 'D'): 0.00798107234882506,
 ('D', 'H'): 0.3300246091661935,
 ('H', 'H'): 0.4367162889775544,
 ('V', 'H'): 0.009949877590409516,
 ('A', 'H'): 0.22330922426584263,
 ('D', 'V'): 0.21777153754047918,
 ('H', 'V'): 0.004738666412861399,
 ('V', 'V'): 0.4896997668219239,
 ('A', 'V'): 0.28779002922473557}

In [56]:
def getQBER(transmitted_states, recieved_states, a_values, b_values):
  cond_probs = {}
  for a in a_values:
    for b in b_values:
      cond_probs[(b, a)] = cond_prob(transmitted_states, recieved_states, a, b)
      
  qber = sum(cond_probs.values())
      
  return cond_probs, qber

In [58]:
a_values = ['D', 'H', 'V']
b_values = ['D', 'H', 'V', 'A']

# QBER achieved in the state H, prob(a!=b|a=H)
a_val = a_values[1]
b_val = [b for b in b_values if b != a_val]

cond_probs, qber = getQBER(transmitted_states_1, recieved_states_1, a_val, b_val)
print(f'P(a!=b|a=H) = {qber}')

# QBER achieved in the state V, prob(a!=b|a=V)
a_val = a_values[2]
b_val = [b for b in b_values if b != a_val]

cond_probs, qber = getQBER(transmitted_states_1, recieved_states_1, a_val, b_val)
print(f'P(a!=b|a=V) = {qber}')

# QBER achieved in the state D, prob(a!=b|a=D)
a_val = a_values[0]
b_val = [b for b in b_values if b != a_val]

cond_probs, qber = getQBER(transmitted_states_1, recieved_states_1, a_val, b_val)
print(f'P(a!=b|a=D) = {qber}')

# QBER achieved in the base X, prob(a!=b|A=X, B=X)
a_X = ['D']
b_X = ['A', 'D']
a_val = a_X[0]
b_val = [b for b in b_X if b != a_val]

cond_probs, qber = getQBER(transmitted_states_1, recieved_states_1, a_val, b_val)
print(f'P(a!=b|A=X, B=X) = {qber}')

# QBER achieved in the base Z, prob(a!=b|A=Z, B=Z)
a_Z = ['H', 'V']
b_Z = ['H', 'V']

a_val = a_Z[0]
b_val = [b for b in b_Z if b != a_val]
cond_probs1, qber1 = getQBER(transmitted_states_1, recieved_states_1, a_val, b_val)

a_val = a_Z[1]
b_val = [b for b in b_Z if b != a_val]
cond_probs2, qber2 = getQBER(transmitted_states_1, recieved_states_1, a_val, b_val)
print(f'P(a!=b|A=Z, B=Z) = {qber1 + qber2}')

P(a!=b|a=H) = 0.5632837110224457
P(a!=b|a=V) = 0.5103002331780762
P(a!=b|a=D) = 0.45365005902040156
P(a!=b|A=X, B=X) = 0.00798107234882506
P(a!=b|A=Z, B=Z) = 0.014688544003270914


**Decoy state**

In [60]:
decoy_high_1 = decoy_1[decoy_1 == 'H']
decoy_low_1 = decoy_1[decoy_1 == 'L']

In [61]:
prob_decoy_high = len(decoy_high_1) / len(decoy_1)
prob_decoy_low = len(decoy_low_1) / len(decoy_1)

print(f'Probability of high intensity state = {prob_decoy_high}')
print(f'Probability of low intensity state = {prob_decoy_low}')
print(f'Total probability = {prob_decoy_high + prob_decoy_low}')

Probability of high intensity state = 0.8891890511645666
Probability of low intensity state = 0.11081094883543337
Total probability = 1.0
